In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My Drive/Homework1

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers as tfkl
import matplotlib.pyplot as plt
from random import randint

In [ ]:
def load_data(folder="public_data.npz", resolution=96, head_only=False):
    images = []

    loaded = np.load(folder, allow_pickle=True)

    # Iterate through files in the specified folder
    for i, img in enumerate(loaded['data']):
        # Normalize image pixel values to a float range [0, 1]
        img = (img / 255).astype(np.float32)

        # Convert image from BGR to RGB
        img = img[...,::-1]

        # Make the image dataset squared
        dim = min(img.shape[:-1])
        img = img[(img.shape[0]-dim)//2:(img.shape[0]+dim)//2, (img.shape[1]-dim)//2:(img.shape[1]+dim)//2, :]

        # Resize the image to 224x224 pixels
        #img = tfkl.Resizing(224, 224)(img)
        img = tfkl.Resizing(resolution, resolution)(img)

        if img is not None:
            images.append(img)

        if (head_only and i == 9):
           break

    labels = loaded['labels']
    loaded.close()

    if (head_only):
       labels = labels[:10]

    y = LabelEncoder().fit_transform(labels)
    y = tfk.utils.to_categorical(y, 2)

    return np.array(images), y




def display_random_images(X, y, num_img=10):
  # Create subplots for displaying items
  fig, axes = plt.subplots(2, num_img//2, figsize=(20, 9))
  for i in range(num_img):
      image = randint(0, X.shape[0] - 1)

      ax = axes[i%2, i%num_img//2]
      ax.imshow(np.clip(X[image], 0, 255))  # Display clipped item images
      ax.text(0.5, -0.1, str(image) + ' ' + str(y[image]), size=12, ha="center", transform=ax.transAxes)
      ax.axis('off')
  plt.tight_layout()
  plt.show()




def delete_outliers(X, y):
  shrek = 137
  trololo = 5143

  new_X = []
  new_y = []
  outliers_X = []

  num_outliers = 0

  for i, sample in enumerate(X):
    if (not (np.array_equal(sample, X[shrek]) or np.array_equal(sample, X[trololo]))):
      new_X.append(sample)
      new_y.append(y[i])
    else:
      outliers_X.append(sample)
      num_outliers += 1

  return np.array(new_X), np.array(new_y), np.array(outliers_X), num_outliers

In [ ]:
X, y = load_data('public_data.npz')
X, y, out, num_outliers = delete_outliers(X, y)

In [ ]:
# Split data into train_val and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=800, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=800, stratify=np.argmax(y_train_val,axis=1))

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)

In [ ]:
augment = tf.keras.Sequential([
    tfkl.RandomFlip(),
    tfkl.RandomTranslation(height_factor = (-0.5,0.5), width_factor = (-0.5,0.5), fill_mode = 'reflect'),
    tfkl.RandomZoom(0.2, fill_mode = 'reflect'),
    tfkl.RandomBrightness(0.2, value_range=(0,1)),
    tfkl.RandomRotation((-1,1), fill_mode = 'reflect'),
])

new_X_train_1 = augment(X_train[np.where((y_train[:, 0] == 0) & (y_train[:, 1] == 1))])
new_X_train_2 = augment(new_X_train_1)
augmented_X_train_1 = augment(X_train)
augmented_X_train_2 = augment(augmented_X_train_1)


X_train = np.append(X_train ,augmented_X_train_1, axis = 0)
X_train = np.append(X_train ,augmented_X_train_2, axis = 0)
X_train = np.append(X_train ,new_X_train_1, axis = 0)
X_train = np.append(X_train ,new_X_train_2, axis = 0)
y_train = np.append(y_train, y_train, axis = 0)
y_train = np.append(y_train, y_train, axis = 0)
for k in range(2*new_X_train_1.shape[0]):
    y_train = np.append(y_train, [[0,1]], axis = 0)

In [ ]:
input_shape = X.shape[1:]
latent_dim = 16

In [ ]:
seed = 42
import os
import random

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
def get_encoder(enc_input_shape=input_shape, enc_output_shape=latent_dim, seed=seed):
    tf.random.set_seed(seed)
    input_layer = tfkl.Input(shape=enc_input_shape, name='input_layer')
    x = tfkl.ZeroPadding2D((2,2))(input_layer)

    x = tfkl.Conv2D(64, 5, padding='same', strides = 3)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.ReLU()(x)
    x = tfkl.Dropout(rate = 1/6)(x)

    x = tfkl.Conv2D(128, 7, padding='same', strides = 4)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.ReLU()(x)
    x = tfkl.Dropout(rate = 1/6)(x)

    x = tfkl.Conv2D(256, 5, padding='same', strides = 3)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.ReLU()(x)
    x = tfkl.Dropout(rate = 1/6)(x)

    x = tfkl.Conv2D(512, 3, padding='same', strides = 2)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.ReLU()(x)
    x = tfkl.Dropout(rate = 1/6)(x)

    x = tfkl.GlobalAveragePooling2D()(x)
    output_layer = tfkl.Dense(enc_output_shape, name='output_layer')(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='encoder')

    # Return the discriminator
    return model

In [ ]:
encoder = get_encoder(input_shape)
encoder.summary()

In [ ]:
def get_decoder(dec_input_shape=latent_dim, dec_output_shape=input_shape, seed=seed):
    tf.random.set_seed(seed)
    input_layer = tfkl.Input(shape=dec_input_shape, name='input_layer')
    x = tfkl.Dense(6*6*256)(input_layer)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.ReLU()(x)
    x = tfkl.Dropout(rate = 1/6)(x)
    x = tfkl.Reshape((6,6,256))(x)

    x = tfkl.Conv2DTranspose(128, 3, padding='same', strides = 2)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.ReLU()(x)
    x = tfkl.Dropout(rate = 1/6)(x)
    x = tfkl.Reshape((12,12,128))(x)

    x = tfkl.Conv2DTranspose(64, 3, padding='same', strides = 2)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.ReLU()(x)
    x = tfkl.Dropout(rate = 1/6)(x)
    x = tfkl.Reshape((24,24,64))(x)

    x = tfkl.Conv2DTranspose(32, 5, padding='same', strides = 2)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.ReLU()(x)
    x = tfkl.Dropout(rate = 1/6)(x)
    x = tfkl.Reshape((48,48,32))(x)

    x = tfkl.Conv2DTranspose(32, 5, padding='same', strides = 2)(x)
    x = tfkl.BatchNormalization()(x)
    x = tfkl.ReLU()(x)
    x = tfkl.Dropout(rate = 1/6)(x)
    x = tfkl.Reshape((96,96,32))(x)

    x = tfkl.Conv2D(dec_output_shape[-1], 5, padding='same')(x)
    x = tfkl.Activation('sigmoid')(x)
    output_layer = tfkl.Cropping2D((0,0))(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='decoder')

    # Return the discriminator
    return model
decoder = get_decoder()
decoder.summary()

In [ ]:
def get_autoencoder(ae_input_shape=input_shape, ae_output_shape=input_shape):
    tf.random.set_seed(seed)

    encoder = get_encoder()
    decoder = get_decoder()

    input_layer = tfkl.Input(shape=ae_input_shape)
    z = encoder(input_layer)
    output_layer = decoder(z)

    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='autoencoder')
    return model
autoencoder = get_autoencoder()
autoencoder.summary()

In [ ]:
learning_rate = 3e-3
optimizer = tf.optimizers.AdamW(learning_rate, weight_decay = 5e-4)
autoencoder.compile(optimizer=optimizer, loss=tfk.losses.binary_crossentropy, metrics=['mse', 'mae'])

batch_size = 16
epochs = 300

In [ ]:
history = autoencoder.fit(
    X_train,
    X_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_val,X_val),
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.9, min_lr=1e-5),
    ]
).history

In [ ]:
def get_reconstructions(model, X, imgs=10, verbose=True):
    predictions = model.predict(X, verbose=0)
    fig, axs = plt.subplots(2, imgs, figsize=(imgs*2, 4))
    for i in range(imgs):
        axs[0, i].imshow(np.squeeze(X[i]), cmap=plt.get_cmap('gray'))
        axs.flat[i].axis('off')
        axs[1, i].imshow(np.squeeze(predictions[i]), cmap=plt.get_cmap('gray'))
        axs.flat[i+imgs].axis('off')
    axs[0,imgs//2].set_title('Real data')
    axs[1,imgs//2].set_title('Reconstructions')
    plt.show()

In [ ]:
get_reconstructions(autoencoder, X_train)